In [1]:
import pandas as pd
import timeit
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

c:\Users\Joseph Dania\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
class Node:
    def __init__(self):
        self.children = {}
        self.end_word = False

class Trie:
    def __init__(self):
        self.root = Node()

    def insert(self, word):
        node = self.root
        for char in word:
            if char not in node.children:
                node.children[char] = Node()
            node = node.children[char]
        node.end_word = True

    def search(self, word):
        node = self.root
        for char in word:
            if char not in node.children:
                return False
            node = node.children[char]
        return node.end_word

    def autocomplete(self, prefix):
        words = []
        node = self.root
        for char in prefix:
            if char not in node.children:
                return words
            node = node.children[char]

        def dfs(current_node, path):
            if current_node.end_word:
                words.append("".join(path))
            for c, child in current_node.children.items():
                dfs(child, path + [c])

        dfs(node, list(prefix))
        return words

In [ ]:

data = pd.read_csv("raw_code.csv")
function_calls = data["Function Calls"].dropna().astype(str)

unique_functions = set() 
for funcs in function_calls:
    for f in funcs.split(","):
        f = f.strip()
        if f:
            unique_functions.add(f)
            
words = list(unique_functions)
print(f"Loaded {len(words)} unique functions.")

trie = Trie()
for word in words:
    trie.insert(word)
print("Trie populated successfully.")

Loaded 3279 unique functions.
Trie populated successfully.


In [ ]:

print("Generating embeddings... (this might take a minute)")
embeddings = model.encode(words)

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

print(f"FAISS Index built with {index.ntotal} vectors.")

def search_ai(query, top_k=5):
    query_vector = model.encode([query])

    distances, indices = index.search(query_vector, top_k)

    results = []
    for idx in indices[0]:
        results.append(words[idx])
        
    return results

Generating embeddings... (this might take a minute)
FAISS Index built with 3279 vectors.


In [ ]:


query = "calculate"


trie_results = trie.autocomplete(query)
print(f"Trie Results (Exact Prefix): {trie_results}")

ai_results = search_ai(query)
print(f"AI Results (Semantic): {ai_results}")

--- SEARCHING FOR: 'calculate' ---

Trie Results (Exact Prefix): ['calculate_pi', 'calculate_probabilities', 'calculate_prime_numbers', 'calculate_signal_power', 'calculate_each_score', 'calculate_variance', 'calculate_fft', 'calculate_current_element', 'calculate_mean', 'calculate_hypothesis_value']
AI Results (Semantic): ['_calculate', 'compute', 'Formula', 'multiply', 'calculate_pi']


In [11]:
search_term = "get"

print("Timing Trie Autocomplete:")
%timeit trie.autocomplete(search_term)

print("Timing AI Search:")
%timeit search_ai(search_term)

Timing Trie Autocomplete:
864 μs ± 272 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
Timing AI Search:
31.1 ms ± 7.53 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:

query = "find someone"
trie_results = trie.autocomplete(query)
print(f"Trie Results (Exact Prefix): {trie_results}")

ai_results = search_ai(query)
print(f"AI Results (Semantic): {ai_results}")

NameError: name 'search_ai' is not defined